# Dice black jack king of the hill controller implementation

* Menší pozitivní rozdíl do 21 vyhrává (negativní rozdíl má nejvyšší možnou penalizaci)
* Při stejném rozdílu vyhrává dealer
* Dealer - nemůže hrát přes 17
* Bettor - nemůže hrát přes 21
* První hraje Bettor


## Třída reprezentující hráče

In [2]:
import random

class Player():
    
    def __init__(self, log, name,
                bettor_handler,
                dealer_handler,
                bet_handler,
                check_handler,
                ):
        self.log = log
        self.name = name
        self.wallet = 1000
        self.value = 0
        self.bettor_handler = bettor_handler
        self.dealer_handler = dealer_handler
        self.bet_handler = bet_handler
        self.check_handler = check_handler
   
    def roll_dice(self):
        return random.randint(1, 6)

    def get_penalty(self):
        penalty = 21 - self.value
        return 100 if penalty < 0 else penalty
    
    def new_round(self):
        self.value = 0
    
    def bettor_play(self):
        still_playing = True
        while still_playing:
            self.value += self.roll_dice()
            still_playing = self.bettor_handler(self.value, self.log)
            if self.value >= 21:
                still_playing = False

    def dealer_play(self):
        still_playing = True
        while still_playing:
            self.value += self.roll_dice()
            still_playing = self.dealer_handler(self.value, self.log)
            if self.value >= 17:
                still_playing = False
    
    def bet(self):
        return self.bet_handler(self.value, self.log)
    
    def check(self, bet):
        return self.check_handler(self.value, self.log, bet)

def dummy_bettor_handler(value, log):
    return True

def dummy_dealer_handler(value, log):
    return True

def dummy_bet_handler(value, log):
    return 5

def dummy_check_handler(value, log, bet):
    return True  

## Simulace hry

In [11]:
def game_round(dealer, bettor, log):
    bettor.new_round()
    dealer.new_round()
    bettor.bettor_play()
    bet = bettor.bet()
    dealer.dealer_play()
    if dealer.check(bet):
        if bettor.get_penalty() < dealer.get_penalty():
            bettor.wallet += bet
            dealer.wallet -= bet
            log.append((bettor.name, "B", bet, 1, bettor.value, dealer.value))
        else:
            bettor.wallet -= bet
            dealer.wallet += bet
            log.append((dealer.name, "D", bet, 1, dealer.value, bettor.value))
    else:
        bettor.wallet += 1
        dealer.wallet -= 1
        log.append((bettor.name, "B", bet, 0, bettor.value, dealer.value))

def simulation(players, n, log):
    for k in range(n):
        game_round(players[0], players[1], log)
        players = players[::-1]
    for p in players:
        print("{}\t{}".format(p.name, p.wallet))
    output = "name,role,bet,check,winner value, loser value\n"
    output += "\n".join([",".join(map(str, line)) for line in log])
    with open("data/output.csv", "w") as f:
        f.write(output)       
    
log = []

p1 = Player(log, "A", dummy_bettor_handler, dummy_dealer_handler,
           dummy_bet_handler, dummy_check_handler)
p2 = Player(log, "B", dummy_bettor_handler, dummy_dealer_handler,
           dummy_bet_handler, dummy_check_handler)

simulation([p1, p2], 100, log)

A	990
B	1010


## Ukázková hra

In [100]:
def better_bettor_handler(value, log):
    if value < 18:
        return True
    else:
        return False

def better_dealer_handler(value, log):
    return True

def better_bet_handler(value, log):
    return random.randint(1,5)

def better_check_handler(value, log, bet):
    if value > 21:
        return False
    else:
        return True  
    
    
log = []
p1 = Player(log, "Better player", better_bettor_handler, better_dealer_handler,
           better_bet_handler, better_check_handler)
p2 = Player(log, "Dummy player", dummy_bettor_handler, dummy_dealer_handler,
           dummy_bet_handler, dummy_check_handler)

simulation([p1, p2], 100, log)

Better player	1097
Dummy player	903


In [101]:
log

[('Better player', 'D', 5, 1, 17, 25),
 ('Dummy player', 'D', 1, 1, 17, 23),
 ('Better player', 'D', 5, 1, 18, 24),
 ('Dummy player', 'D', 1, 1, 21, 21),
 ('Better player', 'D', 5, 1, 19, 22),
 ('Dummy player', 'D', 1, 1, 22, 22),
 ('Dummy player', 'B', 5, 0, 21, 22),
 ('Dummy player', 'D', 4, 1, 17, 23),
 ('Better player', 'D', 5, 1, 20, 26),
 ('Better player', 'B', 1, 1, 20, 18),
 ('Dummy player', 'B', 5, 0, 24, 22),
 ('Better player', 'B', 1, 1, 21, 19),
 ('Better player', 'D', 5, 1, 20, 26),
 ('Dummy player', 'D', 5, 1, 19, 18),
 ('Better player', 'D', 5, 1, 17, 23),
 ('Dummy player', 'D', 2, 1, 18, 18),
 ('Better player', 'D', 5, 1, 18, 25),
 ('Better player', 'B', 3, 1, 19, 17),
 ('Better player', 'D', 5, 1, 20, 22),
 ('Better player', 'B', 4, 1, 19, 18),
 ('Dummy player', 'B', 5, 1, 21, 17),
 ('Dummy player', 'D', 2, 1, 20, 18),
 ('Better player', 'D', 5, 1, 19, 23),
 ('Better player', 'B', 4, 1, 21, 18),
 ('Dummy player', 'B', 5, 1, 21, 18),
 ('Better player', 'B', 3, 1, 20, 19